# Side Task: Evaluacion Integral con Modeva (Lending Club Risk)

Este notebook es una tarea lateral (`side task`) para evaluar metodos y tecnicas de `modeva`
sobre el caso de riesgo de credito del proyecto.

Alcance:
- Ejecutar metodos utiles/relevantes para datos tabulares de default.
- Comparar resultados con el baseline core del proyecto.
- Producir insights tecnicos y recomendacion de adopcion.

Nota:
- No reemplaza el pipeline canonico actual (PD + calibracion + conformal + optimizacion).
- Debe ejecutarse en el entorno principal del proyecto (no en `Legacy`).

In [1]:
import json
import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

warnings.filterwarnings("ignore")

import modeva
import modeva.auth as modeva_auth
from modeva import models

# Evita prompt interactivo de auth en ejecucion batch.
modeva_auth.Authenticator.run = lambda self, auth_code=None: None

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_DIR = PROJECT_ROOT / "data" / "processed"
MODEL_DIR = PROJECT_ROOT / "models"
REPORT_DIR = PROJECT_ROOT / "reports"

print("modeva version:", modeva.__version__)
print("project root:", PROJECT_ROOT)

modeva version: 1.0.15
project root: C:\Users\carlos\Documents\Claude Code\lending-club-risk-project


## 1) Contexto baseline del proyecto core

In [2]:
with open(MODEL_DIR / "pd_training_record.pkl", "rb") as f:
    pd_record = pickle.load(f)

conformal_status = json.loads((MODEL_DIR / "conformal_policy_status.json").read_text())
causal_rule = json.loads((MODEL_DIR / "causal_policy_rule.json").read_text())
ifrs9_summary = pd.read_parquet(DATA_DIR / "ifrs9_scenario_summary.parquet")

core_snapshot = pd.DataFrame(
    {
        "metric": [
            "core_pd_auc_test",
            "core_pd_brier_test",
            "core_conformal_cov90",
            "core_conformal_cov95",
            "core_conformal_min_group_cov90",
            "core_causal_selected_rule",
            "core_ifrs9_baseline_ecl",
            "core_ifrs9_severe_ecl",
        ],
        "value": [
            pd_record["final_test_metrics"]["auc_roc"],
            pd_record["final_test_metrics"]["brier_score"],
            conformal_status["coverage_90"],
            conformal_status["coverage_95"],
            conformal_status["min_group_coverage_90"],
            causal_rule["selected_rule"],
            float(ifrs9_summary.loc[ifrs9_summary["scenario"] == "baseline", "total_ecl"].iloc[0]),
            float(ifrs9_summary.loc[ifrs9_summary["scenario"] == "severe", "total_ecl"].iloc[0]),
        ],
    }
)
core_snapshot

,metric,value
0,core_pd_auc_test,0.718669
1,core_pd_brier_test,0.153747
2,core_conformal_cov90,0.919731
3,core_conformal_cov95,0.960758
4,core_conformal_min_group_cov90,0.891626
5,core_causal_selected_rule,high_plus_medium_positive
6,core_ifrs9_baseline_ecl,795892415.71878
7,core_ifrs9_severe_ecl,1358216518.25118


## 2) Carga de datos y construccion de datasets

In [3]:
base_features = [
    "loan_amnt",
    "annual_inc",
    "dti",
    "int_rate",
    "installment",
    "loan_to_income",
    "rev_utilization",
    "open_acc",
    "total_acc",
    "fico_score",
    "grade",
    "home_ownership",
    "purpose",
    "verification_status",
    "term",
]
target_col = "default_flag"

train_raw_full = pd.read_parquet(DATA_DIR / "train_fe.parquet")
test_raw_full = pd.read_parquet(DATA_DIR / "test_fe.parquet")

raw_cols = base_features + [target_col]
train_raw = train_raw_full.sample(6000, random_state=RANDOM_STATE).reset_index(drop=True)[raw_cols].copy()
test_raw = test_raw_full.sample(3000, random_state=RANDOM_STATE).reset_index(drop=True)[raw_cols].copy()

protected_data = pd.concat(
    [train_raw[["home_ownership"]], test_raw[["home_ownership"]]],
    axis=0,
).reset_index(drop=True)

combined = pd.concat([train_raw, test_raw], axis=0, ignore_index=True)
combined_num = pd.get_dummies(
    combined,
    columns=["grade", "home_ownership", "purpose", "verification_status"],
    drop_first=True,
)
combined_num = combined_num.fillna(combined_num.median(numeric_only=True))

train_num = combined_num.iloc[: len(train_raw)].reset_index(drop=True)
test_num = combined_num.iloc[len(train_raw) :].reset_index(drop=True)

print("train_raw:", train_raw.shape, "test_raw:", test_raw.shape)
print("train_num:", train_num.shape, "test_num:", test_num.shape)
print("train default rate:", round(train_raw[target_col].mean(), 6))
print("test default rate:", round(test_raw[target_col].mean(), 6))

train_raw: (6000, 16) test_raw: (3000, 16)
train_num: (6000, 38) test_num: (3000, 38)
train default rate: 0.184167
test default rate: 0.242667


## 3) Setup Modeva + helpers

In [4]:
ds_raw = modeva.DataSet(name="lc_modeva_raw_side_task")
ds_raw.load_dataframe_train_test(train_raw, test_raw)
ds_raw.set_target(target_col)

ds = modeva.DataSet(name="lc_modeva_numeric_side_task")
ds.load_dataframe_train_test(train_num, test_num)
ds.set_target(target_col)
ds.set_protected_data(protected_data)

results = {}
run_log = []


def run_test(name, fn):
    try:
        out = fn()
        results[name] = out
        run_log.append({"test": name, "status": "ok", "error": ""})
        table = getattr(out, "table", None)
        if isinstance(table, pd.DataFrame):
            print(f"[OK] {name}: table_shape={table.shape}")
        elif isinstance(table, dict):
            print(f"[OK] {name}: table_keys={list(table.keys())}")
        else:
            print(f"[OK] {name}: no_table")
        return out
    except Exception as e:
        run_log.append({"test": name, "status": "error", "error": repr(e)})
        print(f"[FAIL] {name}: {repr(e)}")
        return None


def result_table(name):
    out = results.get(name)
    if out is None:
        return None
    table = getattr(out, "table", None)
    return table if isinstance(table, pd.DataFrame) else None


def result_value(name):
    out = results.get(name)
    if out is None:
        return None
    return getattr(out, "value", None)

## 4) Preprocessing nativo de Modeva

In [5]:
_ = ds_raw.impute_missing(add_indicators=True)
_ = ds_raw.encode_categorical(method="onehot")
_ = ds_raw.scale_numerical(method="standardize")
_ = ds_raw.bin_numerical(features=("int_rate", "dti"), method="quantile", bins=5)
ds_raw.preprocess()

preprocess_summary = {
    "raw_shape": tuple(ds_raw.to_df(raw_data=True).shape),
    "preprocessed_shape": tuple(ds_raw.to_df(raw_data=False).shape),
}

x_small = ds_raw.get_data("train")[:10, :-1]
try:
    _ = ds_raw.transform(x_small)
    preprocess_summary["transform_status"] = "ok"
except Exception as e:
    preprocess_summary["transform_status"] = f"error: {repr(e)}"

try:
    _ = ds_raw.inverse_transform(x_small)
    preprocess_summary["inverse_transform_status"] = "ok"
except Exception as e:
    preprocess_summary["inverse_transform_status"] = f"error: {repr(e)}"

pd.DataFrame([preprocess_summary])

,raw_shape,preprocessed_shape,transform_status,inverse_transform_status
0,"(9000, 16)","(9000, 41)",error: ValueError('Shape of passed values is (...,error: ValueError('Shape of passed values is (...


## 5) Data diagnostics: EDA, drift, outliers, feature selection

In [6]:
run_test("summary", lambda: ds.summary())
run_test("eda_1d", lambda: ds.eda_1d(feature="int_rate", dataset="train", sample_size=2000))
run_test(
    "eda_2d",
    lambda: ds.eda_2d(
        feature_x="int_rate",
        feature_y="loan_to_income",
        feature_color=target_col,
        dataset="train",
        sample_size=2000,
    ),
)
run_test(
    "eda_3d",
    lambda: ds.eda_3d(
        feature_x="int_rate",
        feature_y="loan_to_income",
        feature_z="fico_score",
        feature_color=target_col,
        dataset="train",
        sample_size=1000,
    ),
)
run_test(
    "eda_correlation",
    lambda: ds.eda_correlation(
        features=("int_rate", "loan_to_income", "fico_score", "dti", target_col),
        dataset="train",
    ),
)
run_test("eda_pca", lambda: ds.eda_pca(dataset="train", sample_size=2000))
run_test(
    "eda_umap",
    lambda: ds.eda_umap(dataset="train", sample_size=1000, n_neighbors=15, random_state=RANDOM_STATE),
)

run_test(
    "drift_psi",
    lambda: ds.data_drift_test(dataset1="train", dataset2="test", distance_metric="PSI", psi_bins=10),
)
run_test("drift_wd1", lambda: ds.data_drift_test(dataset1="train", dataset2="test", distance_metric="WD1"))
run_test("drift_ks", lambda: ds.data_drift_test(dataset1="train", dataset2="test", distance_metric="KS"))

run_test(
    "outlier_iforest",
    lambda: ds.detect_outlier_isolation_forest(dataset="train", threshold=0.995, n_estimators=200),
)
run_test(
    "outlier_pca",
    lambda: ds.detect_outlier_pca(dataset="train", threshold=0.995, method="mahalanobis"),
)
run_test(
    "outlier_cblof",
    lambda: ds.detect_outlier_cblof(dataset="train", threshold=0.995, n_clusters=12, method="kmeans"),
)

run_test("fs_corr", lambda: ds.feature_select_corr(dataset="train", threshold=0.80))
run_test("fs_xgbpfi", lambda: ds.feature_select_xgbpfi(dataset="train", threshold=0.01))
run_test(
    "fs_rcit",
    lambda: ds.feature_select_rcit(
        dataset="train", threshold=1e-5, n_fourier=10, n_fourier2=3, n_forwards=1, random_state=RANDOM_STATE
    ),
)
run_test(
    "subsample_random",
    lambda: ds.subsample_random(dataset="train", sample_size=0.25, stratify=target_col, random_state=RANDOM_STATE),
)

[OK] summary: table_keys=['summary', 'numerical', 'categorical', 'mixed', 'date']
[OK] eda_1d: no_table
[OK] eda_2d: no_table
[OK] eda_3d: no_table
[OK] eda_correlation: table_shape=(5, 5)


[OK] eda_pca: table_shape=(2000, 38)


[OK] eda_umap: table_shape=(1000, 2)


[OK] drift_psi: table_shape=(38, 1)


[OK] drift_wd1: table_shape=(38, 1)


[OK] drift_ks: table_shape=(38, 1)


[OK] outlier_iforest: table_keys=['outliers', 'non-outliers']
[OK] outlier_pca: table_keys=['outliers', 'non-outliers']


[OK] outlier_cblof: table_keys=['outliers', 'non-outliers']
[OK] fs_corr: table_shape=(37, 3)


[OK] fs_xgbpfi: table_shape=(37, 3)


[OK] fs_rcit: table_shape=(37, 1)
[OK] subsample_random: no_table


In [7]:
display(Markdown("### Top drift (PSI)"))
psi_table = result_table("drift_psi")
if psi_table is not None:
    display(psi_table.sort_values("Distance_Scores", ascending=False).head(10))

display(Markdown("### Feature selection (XGB-PFI top)"))
fs_xgb = result_table("fs_xgbpfi")
if fs_xgb is not None:
    display(fs_xgb.sort_values("Importance", ascending=False).head(10))

### Top drift (PSI)

,Distance_Scores
rev_utilization,0.261071
loan_to_income,0.156510
fico_score,0.155928
loan_amnt,0.114104
verification_status_Verified,0.074746
int_rate,0.065613
dti,0.058955
installment,0.038461
grade_E,0.025765
grade_F,0.023243


### Feature selection (XGB-PFI top)

,Name,Importance,Selected
3,int_rate,0.150161,True
4,installment,0.116997,True
2,dti,0.101681,True
5,loan_to_income,0.087978,True
6,rev_utilization,0.076808,True
8,total_acc,0.072087,True
1,annual_inc,0.070244,True
0,loan_amnt,0.062414,True
9,fico_score,0.056426,True
7,open_acc,0.039959,True


## 6) ModelZoo: entrenamiento y tuning

In [8]:
mz = modeva.ModelZoo(dataset=ds, name="lc_modeva_side_task_zoo", random_state=RANDOM_STATE)
mz.add_model(models.MoLogisticRegression(max_iter=500), name="MoLogReg", replace=True)
mz.add_model(models.MoLGBMClassifier(n_estimators=300, max_depth=6, verbose=-1), name="MoLGBM", replace=True)
mz.add_model(
    models.MoXGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.05, verbosity=0),
    name="MoXGB",
    replace=True,
)
mz.train_all(silent=True)
leaderboard = mz.leaderboard(order_by="test AUC", ascending=False)
leaderboard

,start_time,end_time,Duration,train AUC,test AUC,train ACC,test ACC,train F1,test F1,train LogLoss,test LogLoss,train Brier,test Brier
MoLogReg,2026-02-09 10:00:35,2026-02-09 10:00:35,0.237958,0.699088,0.669238,0.816333,0.757333,0.092257,0.110024,0.439247,0.55266,0.138404,0.177486
MoXGB,2026-02-09 10:00:35,2026-02-09 10:00:36,0.217495,0.93332,0.662946,0.873833,0.751333,0.485384,0.107656,0.300563,0.563194,0.089187,0.181785
MoLGBM,2026-02-09 10:00:35,2026-02-09 10:00:35,0.136181,0.99722,0.640788,0.960667,0.749667,0.880688,0.14562,0.165017,0.639588,0.039527,0.193847


In [9]:
tuner = models.ModelTuneRandomSearch(ds, models.MoLogisticRegression(max_iter=500))
run_test(
    "tune_random_logreg",
    lambda: tuner.run(
        param_distributions={"C": (0.01, 5.0)},
        dataset="train",
        n_iter=5,
        metric="AUC",
        cv=3,
        random_state=RANDOM_STATE,
    ),
)
result_table("tune_random_logreg")

[OK] tune_random_logreg: table_shape=(2, 4)


,C,AUC,AUC_rank,Time
0,0.01,0.696517,1,0.207504
1,5.00,0.695939,2,0.209799


## 7) TestSuite: calidad, robustez, reliability, fairness y slicing

In [10]:
model_primary = mz.get_model("MoLGBM")
model_list = [mz.get_model(n) for n in mz.list_model_names()]
ts = modeva.TestSuite(dataset=ds, model=model_primary, models=model_list, name="lc_modeva_side_task_suite")

group_cfg = {
    "rent_vs_mortgage": {
        "feature": "home_ownership",
        "protected": "RENT",
        "reference": "MORTGAGE",
    }
}

run_test("diag_accuracy", lambda: ts.diagnose_accuracy_table())
run_test("cmp_accuracy", lambda: ts.compare_accuracy_table())
run_test("diag_reliability", lambda: ts.diagnose_reliability(train_dataset="train", test_dataset="test", alpha=0.1, width_threshold=0.15))
run_test("cmp_reliability", lambda: ts.compare_reliability(train_dataset="train", test_dataset="test", alpha=0.1))
run_test("diag_robustness", lambda: ts.diagnose_robustness(dataset="test", perturb_features=("int_rate", "dti"), noise_levels=(0.02, 0.05, 0.1), n_repeats=5))
run_test("cmp_robustness", lambda: ts.compare_robustness(dataset="test", perturb_features=("int_rate", "dti"), noise_levels=(0.02, 0.05, 0.1), n_repeats=5))
run_test("diag_resilience", lambda: ts.diagnose_resilience(dataset="test", method="worst-sample", alphas=(0.1, 0.2, 0.3)))
run_test("cmp_resilience", lambda: ts.compare_resilience(dataset="test", method="worst-sample", alphas=(0.1, 0.2, 0.3)))
run_test("diag_residual_analysis", lambda: ts.diagnose_residual_analysis(features="int_rate", dataset="test", sample_size=1500))
run_test("diag_residual_cluster", lambda: ts.diagnose_residual_cluster(dataset="test", n_clusters=8, sample_size=1500))
run_test("cmp_residual_cluster", lambda: ts.compare_residual_cluster(dataset="test", n_clusters=8, sample_size=1500))
run_test("diag_residual_interpret", lambda: ts.diagnose_residual_interpret(dataset="test", n_estimators=80, max_depth=2))
run_test("diag_fairness", lambda: ts.diagnose_fairness(group_config=group_cfg, dataset="test", metric="AIR"))
run_test("cmp_fairness", lambda: ts.compare_fairness(group_config=group_cfg, dataset="test", metric="AIR"))

run_test("diag_slicing_accuracy", lambda: ts.diagnose_slicing_accuracy(features=("int_rate", "fico_score"), dataset="test", bins=5))
run_test("cmp_slicing_accuracy", lambda: ts.compare_slicing_accuracy(features="int_rate", dataset="test", bins=5))
run_test("diag_slicing_overfit", lambda: ts.diagnose_slicing_overfit(features=("int_rate", "fico_score"), train_dataset="train", test_dataset="test", bins=5))
run_test("cmp_slicing_overfit", lambda: ts.compare_slicing_overfit(features="int_rate", train_dataset="train", test_dataset="test", bins=5))
run_test("diag_slicing_robustness", lambda: ts.diagnose_slicing_robustness(features=("int_rate", "fico_score"), dataset="test", bins=5, perturb_features=("int_rate", "dti"), noise_levels=0.05, n_repeats=3))
run_test("cmp_slicing_robustness", lambda: ts.compare_slicing_robustness(features="int_rate", dataset="test", bins=5, perturb_features=("int_rate", "dti"), noise_levels=0.05, n_repeats=3))
run_test("diag_slicing_reliability", lambda: ts.diagnose_slicing_reliability(features=("int_rate", "fico_score"), train_dataset="train", test_dataset="test", bins=5, alpha=0.1))
run_test("cmp_slicing_reliability", lambda: ts.compare_slicing_reliability(features="int_rate", train_dataset="train", test_dataset="test", bins=5, alpha=0.1))
run_test("diag_slicing_fairness", lambda: ts.diagnose_slicing_fairness(group_config=group_cfg, features=("int_rate", "fico_score"), dataset="test", bins=5, metric="AIR"))
run_test("cmp_slicing_fairness", lambda: ts.compare_slicing_fairness(group_config=group_cfg, features="int_rate", dataset="test", bins=5, metric="AIR"))

run_test("diag_mitigate_unfair_thresholding", lambda: ts.diagnose_mitigate_unfair_thresholding(group_config=group_cfg, dataset="test", metric="AIR", performance_metric="AUC", proba_cutoff=(0.30, 0.35, 0.40, 0.45)))
run_test("diag_mitigate_unfair_binning", lambda: ts.diagnose_mitigate_unfair_binning(group_config=group_cfg, dataset="test", metric="AIR", performance_metric="AUC", binning_features=("int_rate", "fico_score"), bins=5))

[OK] diag_accuracy: table_shape=(3, 5)


[OK] cmp_accuracy: table_shape=(3, 15)
[OK] diag_reliability: table_shape=(1, 2)


[OK] cmp_reliability: table_shape=(1, 6)


[OK] diag_robustness: table_shape=(5, 4)


[OK] cmp_robustness: table_shape=(5, 12)
[OK] diag_resilience: table_shape=(3, 1)
[OK] cmp_resilience: table_shape=(3, 3)
[OK] diag_residual_analysis: table_shape=(1500, 2)


...

[OK] diag_slicing_robustness: table_shape=(25, 8)


[OK] cmp_slicing_robustness: table_shape=(5, 12)
[OK] diag_slicing_reliability: table_shape=(25, 8)


[OK] cmp_slicing_reliability: table_shape=(5, 12)
[OK] diag_slicing_fairness: table_keys=['rent_vs_mortgage']


[OK] cmp_slicing_fairness: table_keys=['rent_vs_mortgage']
[OK] diag_mitigate_unfair_thresholding: table_keys=['Fairness', 'Performance']


[OK] diag_mitigate_unfair_binning: table_keys=['Fairness', 'Performance']


In [11]:
display(Markdown("### Compare Accuracy"))
display(result_table("cmp_accuracy"))
display(Markdown("### Compare Reliability"))
display(result_table("cmp_reliability"))
display(Markdown("### Compare Fairness (AIR)"))
display(result_table("cmp_fairness"))

### Compare Accuracy

MoLogisticRegression                                          \
                       AUC       ACC        F1   LogLoss     Brier   
train             0.699088  0.816333  0.092257  0.439247  0.138404   
test              0.669238  0.757333  0.110024  0.552660  0.177486   
GAP              -0.029850 -0.059000  0.017767  0.113413  0.039082   

      MoLGBMClassifier                                          \
                   AUC       ACC        F1   LogLoss     Brier   
train         0.997220  0.960667  0.880688  0.165017  0.039527   
test          0.640788  0.749667  0.145620  0.639588  0.193847   
GAP          -0.356431 -0.211000 -0.735068  0.474571  0.154320   

      MoXGBClassifier                                          
                  AUC       ACC        F1   LogLoss     Brier  
train        0.933320  0.873833  0.485384  0.300563  0.089187  
test         0.662946  0.751333  0.107656  0.563194  0.181785  
GAP         -0.270375 -0.122500 -0.377729  0.262631  0.092597

### Compare Reliability

MoLogisticRegression              MoLGBMClassifier               \
             Avg.Width Avg.Coverage        Avg.Width Avg.Coverage   
0             1.246333        0.853         0.904667        0.696   

  MoXGBClassifier               
        Avg.Width Avg.Coverage  
0        1.043333     0.773333

### Compare Fairness (AIR)

,MoLogisticRegression,MoLGBMClassifier,MoXGBClassifier
,rent_vs_mortgage,rent_vs_mortgage,rent_vs_mortgage
AIR,1.554291,1.611857,2.149142


## 8) Explainability e interpretabilidad

In [12]:
run_test("explain_pfi", lambda: ts.explain_pfi(dataset="test", sample_size=2000, n_repeats=5))
run_test("explain_shap", lambda: ts.explain_shap(dataset="test", sample_index=25, baseline_sample_size=300))
run_test("explain_lime", lambda: ts.explain_lime(dataset="test", sample_index=25))
run_test("explain_pdp", lambda: ts.explain_pdp(features=("int_rate", "loan_to_income"), dataset="test", sample_size=2000))
run_test("explain_ale", lambda: ts.explain_ale(features=("int_rate", "loan_to_income"), dataset="test", sample_size=2000))
run_test("explain_hstat", lambda: ts.explain_hstatistic(features=("int_rate", "fico_score", "loan_to_income"), dataset="test", sample_size=1500))

ts_logreg = modeva.TestSuite(dataset=ds, model=mz.get_model("MoLogReg"), models=model_list, name="lc_modeva_side_task_suite_logreg")
run_test("interpret_coef", lambda: ts_logreg.interpret_coef())
run_test("interpret_effects", lambda: ts_logreg.interpret_effects())
run_test("interpret_ei", lambda: ts_logreg.interpret_ei(dataset="test"))
run_test("interpret_local_ei", lambda: ts_logreg.interpret_local_ei(dataset="test", sample_index=25))
run_test("interpret_fi", lambda: ts_logreg.interpret_fi(dataset="test"))
run_test("interpret_local_fi", lambda: ts_logreg.interpret_local_fi(dataset="test", sample_index=25))
run_test("interpret_local_linear_fi", lambda: ts_logreg.interpret_local_linear_fi(dataset="test", sample_index=25))

mz_tree = modeva.ModelZoo(dataset=ds, name="lc_modeva_side_task_tree", random_state=RANDOM_STATE)
mz_tree.add_model(models.MoDecisionTreeClassifier(max_depth=4), name="MoTree", replace=True)
mz_tree.train("MoTree")
ts_tree = modeva.TestSuite(dataset=ds, model=mz_tree.get_model("MoTree"), name="lc_modeva_side_task_suite_tree")
run_test("interpret_global_tree", lambda: ts_tree.interpret_global_tree())
run_test("interpret_local_tree", lambda: ts_tree.interpret_local_tree(dataset="test", sample_index=25))

[OK] explain_pfi: table_shape=(37, 2)


[OK] explain_shap: table_shape=(37, 3)
[OK] explain_lime: table_shape=(37, 4)


[OK] explain_pdp: table_shape=(20, 20)
[OK] explain_ale: table_shape=(21, 21)


[OK] explain_hstat: table_shape=(3, 3)


[OK] interpret_coef: table_shape=(37, 2)


[OK] interpret_effects: table_keys=['loan_amnt', 'annual_inc', 'dti', 'int_rate', 'installment', 'loan_to_income', 'rev_utilization', 'open_acc', 'total_acc', 'fico_score', 'term', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F', 'grade_G', 'home_ownership_MORTGAGE', 'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT', 'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_educational', 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy', 'purpose_small_business', 'purpose_vacation', 'purpose_wedding', 'verification_status_Source Verified', 'verification_status_Verified']


[OK] interpret_ei: table_shape=(37, 2)
[FAIL] interpret_local_ei: AttributeError("'LogisticRegression' object has no attribute 'modeva_intercept_'")


[OK] interpret_fi: table_shape=(37, 2)


[OK] interpret_local_fi: table_shape=(37, 3)
[OK] interpret_local_linear_fi: table_shape=(37, 4)


[OK] interpret_global_tree: no_table
[OK] interpret_local_tree: no_table


In [13]:
display(Markdown("### PFI top"))
pfi_table = result_table("explain_pfi")
if pfi_table is not None:
    display(pfi_table.sort_values("Importance", ascending=False).head(15))

display(Markdown("### SHAP local (sample 25) top |effect|"))
shap_table = result_table("explain_shap")
if shap_table is not None:
    display(shap_table.assign(abs_effect=shap_table["Effect"].abs()).sort_values("abs_effect", ascending=False).head(15))

display(Markdown("### Coeficientes MoLogReg"))
coef_table = result_table("interpret_coef")
if coef_table is not None:
    display(coef_table.sort_values("Coefficients", ascending=False).head(15))

### PFI top

,Name,Importance
3,int_rate,0.0047
2,dti,0.0043
21,home_ownership_RENT,0.0036
9,fico_score,0.0034
0,loan_amnt,0.0028
17,home_ownership_MORTGAGE,0.0025
8,total_acc,0.0019
23,purpose_debt_consolidation,0.0013
22,purpose_credit_card,0.0013
10,term,0.0011


### SHAP local (sample 25) top |effect|

,Name,Value,Effect,abs_effect
1,annual_inc,65000.0,0.088301,0.088301
4,installment,247.97,0.034404,0.034404
20,home_ownership_OWN,False,0.028168,0.028168
19,home_ownership_OTHER,False,0.025808,0.025808
18,home_ownership_NONE,False,-0.018238,0.018238
10,term,36,0.016343,0.016343
36,verification_status_Verified,True,0.013860,0.013860
23,purpose_debt_consolidation,True,-0.013033,0.013033
3,int_rate,7.26,0.010310,0.010310
21,home_ownership_RENT,False,-0.006504,0.006504


### Coeficientes MoLogReg

,Name,Coefficients
3,X3,0.088435
7,X7,0.029265
10,X10,0.026536
21,X21,0.009787
12,X12,0.006338
2,X2,0.005343
35,X35,0.004110
14,X14,0.003696
13,X13,0.003157
36,X36,0.002257


## 9) Export reporte Modeva + auditoria de ejecucion

In [14]:
report_path = REPORT_DIR / "modeva_side_task_report.html"
run_test("export_report", lambda: ts.export_report(path=str(report_path)))

run_df = pd.DataFrame(run_log)
print("status counts")
display(run_df["status"].value_counts())
print("report path:", report_path)
run_df.sort_values(["status", "test"]).reset_index(drop=True)

[OK] export_report: no_table
status counts


status
ok       59
error     1
Name: count, dtype: int64

report path: C:\Users\carlos\Documents\Claude Code\lending-club-risk-project\reports\modeva_side_task_report.html


,test,status,error
0,interpret_local_ei,error,"AttributeError(""'LogisticRegression' object ha..."
1,cmp_accuracy,ok,
2,cmp_fairness,ok,
3,cmp_reliability,ok,
4,cmp_residual_cluster,ok,
5,cmp_resilience,ok,
6,cmp_robustness,ok,
7,cmp_slicing_accuracy,ok,
8,cmp_slicing_fairness,ok,
9,cmp_slicing_overfit,ok,


## 10) Insights de resultados y aporte potencial

In [15]:
# Performance por modelo en test
cmp_accuracy = result_table("cmp_accuracy")
model_perf_rows = []
if cmp_accuracy is not None:
    model_names = cmp_accuracy.columns.get_level_values(0).unique()
    for model_name in model_names:
        model_perf_rows.append(
            {
                "model": model_name,
                "test_auc": float(cmp_accuracy.loc["test", (model_name, "AUC")]),
                "test_brier": float(cmp_accuracy.loc["test", (model_name, "Brier")]),
                "gap_auc_test_minus_train": float(cmp_accuracy.loc["GAP", (model_name, "AUC")]),
            }
        )
model_perf = pd.DataFrame(model_perf_rows).sort_values("test_auc", ascending=False)
display(Markdown("### Performance comparada (Modeva sample)"))
display(model_perf)

# Drift top
drift_top = None
psi_table = result_table("drift_psi")
if psi_table is not None:
    drift_top = psi_table.sort_values("Distance_Scores", ascending=False).head(10)
display(Markdown("### Top drift features (PSI)"))
display(drift_top)

# Outlier rates
def outlier_rate(result_name):
    out = results.get(result_name)
    if out is None:
        return np.nan
    table = getattr(out, "table", None)
    if not isinstance(table, dict):
        return np.nan
    n_out = len(table.get("outliers", []))
    n_non = len(table.get("non-outliers", []))
    return n_out / (n_out + n_non) if (n_out + n_non) > 0 else np.nan


outlier_summary = pd.DataFrame(
    {
        "method": ["IsolationForest", "PCA-Mahalanobis", "CBLOF"],
        "outlier_rate": [
            outlier_rate("outlier_iforest"),
            outlier_rate("outlier_pca"),
            outlier_rate("outlier_cblof"),
        ],
    }
)
display(Markdown("### Outlier rate por metodo"))
display(outlier_summary)

# Feature selection consensus
fs_corr = result_table("fs_corr")
fs_xgb = result_table("fs_xgbpfi")
fs_rcit_value = result_value("fs_rcit")
fs_corr_sel = set(fs_corr.loc[fs_corr["Selected"] == True, "Name"].tolist()) if fs_corr is not None else set()
fs_xgb_sel = set(fs_xgb.loc[fs_xgb["Selected"] == True, "Name"].tolist()) if fs_xgb is not None else set()
fs_rcit_sel = set(fs_rcit_value.get("selected", [])) if isinstance(fs_rcit_value, dict) else set()
fs_consensus = sorted((fs_corr_sel & fs_xgb_sel) | fs_rcit_sel)

fs_summary = pd.DataFrame(
    {
        "selected_by_corr": [len(fs_corr_sel)],
        "selected_by_xgbpfi": [len(fs_xgb_sel)],
        "selected_by_rcit": [len(fs_rcit_sel)],
        "consensus_plus_rcit": [", ".join(fs_consensus[:20])],
    }
)
display(Markdown("### Feature selection resumen"))
display(fs_summary)

# Reliability / fairness / robustness summaries
reliability_table = result_table("cmp_reliability")
reliability_rows = []
if reliability_table is not None:
    for model_name in reliability_table.columns.get_level_values(0).unique():
        reliability_rows.append(
            {
                "model": model_name,
                "avg_width": float(reliability_table.loc[0, (model_name, "Avg.Width")]),
                "avg_coverage": float(reliability_table.loc[0, (model_name, "Avg.Coverage")]),
            }
        )
reliability_summary = pd.DataFrame(reliability_rows)
display(Markdown("### Reliability resumen"))
display(reliability_summary)

fairness_table = result_table("cmp_fairness")
fairness_rows = []
if fairness_table is not None:
    for model_name in fairness_table.columns.get_level_values(0).unique():
        fairness_rows.append(
            {
                "model": model_name,
                "AIR_rent_vs_mortgage": float(fairness_table.loc["AIR", (model_name, "rent_vs_mortgage")]),
            }
        )
fairness_summary = pd.DataFrame(fairness_rows)
display(Markdown("### Fairness resumen (AIR)"))
display(fairness_summary)

robustness_table = result_table("cmp_robustness")
robustness_rows = []
if robustness_table is not None:
    robust_grouped = robustness_table.groupby(level=[0, 1], axis=1).mean()
    noise_levels = sorted(set(robust_grouped.columns.get_level_values(1).tolist()))
    max_noise = max(noise_levels)
    for model_name in robust_grouped.columns.get_level_values(0).unique():
        base = float(robust_grouped[(model_name, 0.0)].mean())
        high_noise = float(robust_grouped[(model_name, max_noise)].mean())
        robustness_rows.append(
            {
                "model": model_name,
                "metric_at_noise_0.0": base,
                f"metric_at_noise_{max_noise}": high_noise,
                "delta_high_noise_minus_base": high_noise - base,
            }
        )
robustness_summary = pd.DataFrame(robustness_rows)
display(Markdown("### Robustness resumen"))
display(robustness_summary)

# Compare best Modeva AUC vs core PD AUC
core_auc = float(pd_record["final_test_metrics"]["auc_roc"])
best_modeva_auc = float(model_perf["test_auc"].max()) if not model_perf.empty else np.nan
auc_gap_vs_core = best_modeva_auc - core_auc if not np.isnan(best_modeva_auc) else np.nan

comparison_vs_core = pd.DataFrame(
    [{"core_pd_auc": core_auc, "best_modeva_auc_sample": best_modeva_auc, "modeva_minus_core_auc": auc_gap_vs_core}]
)
display(Markdown("### Comparacion AUC vs baseline core"))
display(comparison_vs_core)

### Performance comparada (Modeva sample)

,model,test_auc,test_brier,gap_auc_test_minus_train
0,MoLogisticRegression,0.669238,0.177486,-0.029850
2,MoXGBClassifier,0.662946,0.181785,-0.270375
1,MoLGBMClassifier,0.640788,0.193847,-0.356431


### Top drift features (PSI)

,Distance_Scores
rev_utilization,0.261071
loan_to_income,0.156510
fico_score,0.155928
loan_amnt,0.114104
verification_status_Verified,0.074746
int_rate,0.065613
dti,0.058955
installment,0.038461
grade_E,0.025765
grade_F,0.023243


### Outlier rate por metodo

...

,model,AIR_rent_vs_mortgage
0,MoLogisticRegression,1.554291
1,MoLGBMClassifier,1.611857
2,MoXGBClassifier,2.149142


### Robustness resumen

,model,metric_at_noise_0.0,metric_at_noise_0.1,delta_high_noise_minus_base
0,MoLGBMClassifier,0.640788,0.638979,-0.001810
1,MoLogisticRegression,0.669238,0.669236,-0.000002
2,MoXGBClassifier,0.662946,0.662314,-0.000631


### Comparacion AUC vs baseline core

,core_pd_auc,best_modeva_auc_sample,modeva_minus_core_auc
0,0.718669,0.669238,-0.049431


In [16]:
failures = run_df.loc[run_df["status"] == "error", ["test", "error"]].copy()

lines = []
lines.append("## Conclusiones de la side task (Modeva)")
lines.append("")
lines.append(f"- Metodos OK: {int((run_df['status'] == 'ok').sum())}")
lines.append(f"- Metodos con error controlado: {int((run_df['status'] == 'error').sum())}")
if not model_perf.empty:
    best = model_perf.iloc[0]
    lines.append(f"- Mejor AUC en muestra Modeva: {best['model']} = {best['test_auc']:.4f}")
    lines.append(f"- Brecha vs AUC core ({core_auc:.4f}): {auc_gap_vs_core:+.4f}")
if drift_top is not None and not drift_top.empty:
    lines.append(f"- Mayor drift PSI: {drift_top.index[0]} ({float(drift_top.iloc[0]['Distance_Scores']):.4f})")
if not fairness_summary.empty:
    lines.append(
        f"- AIR rent_vs_mortgage (min/max): {fairness_summary['AIR_rent_vs_mortgage'].min():.4f} / {fairness_summary['AIR_rent_vs_mortgage'].max():.4f}"
    )
if not failures.empty:
    lines.append("- Limites observados en modeva:")
    for _, row in failures.iterrows():
        lines.append(f"  - {row['test']}: {row['error']}")

lines.append("")
lines.append("Aporte potencial al proyecto:")
lines.append("- Modeva aporta una bateria muy amplia de validaciones de gobernanza (drift, slicing, fairness, robustez, reliability, interpretabilidad).")
lines.append("- En esta muestra no supera el baseline de AUC del modelo canonico; no se recomienda reemplazo del core en este estado.")
lines.append("- Se recomienda uso complementario para auditoria/monitoring del modelo core.")
lines.append("")
lines.append("Esta entrega se considera side task y no parte del core, salvo que en futuras pruebas muestre mejoras materiales.")

display(Markdown("\\n".join(lines)))

## Conclusiones de la side task (Modeva)\n\n- Metodos OK: 59\n- Metodos con error controlado: 1\n- Mejor AUC en muestra Modeva: MoLogisticRegression = 0.6692\n- Brecha vs AUC core (0.7187): -0.0494\n- Mayor drift PSI: rev_utilization (0.2611)\n- AIR rent_vs_mortgage (min/max): 1.5543 / 2.1491\n- Limites observados en modeva:\n  - interpret_local_ei: AttributeError("'LogisticRegression' object has no attribute 'modeva_intercept_'")\n\nAporte potencial al proyecto:\n- Modeva aporta una bateria muy amplia de validaciones de gobernanza (drift, slicing, fairness, robustez, reliability, interpretabilidad).\n- En esta muestra no supera el baseline de AUC del modelo canonico; no se recomienda reemplazo del core en este estado.\n- Se recomienda uso complementario para auditoria/monitoring del modelo core.\n\nEsta entrega se considera side task y no parte del core, salvo que en futuras pruebas muestre mejoras materiales.

## 11) Recomendacion explicita: que llevar al core vs que descartar

In [17]:
method_decisions = [
    {
        "method_group": "data_drift_test (PSI/KS)",
        "status": "RECOMENDADO",
        "decision": "Incluir como monitoreo periodico",
        "justification": "Entrega alerta temprana de cambio distribucional en variables de riesgo.",
    },
    {
        "method_group": "diagnose_fairness / diagnose_slicing_fairness",
        "status": "RECOMENDADO",
        "decision": "Incluir como control de gobernanza",
        "justification": "Detecta sesgos por subgrupo y soporta discusiones de politica de credito.",
    },
    {
        "method_group": "diagnose_robustness / diagnose_slicing_robustness",
        "status": "RECOMENDADO",
        "decision": "Incluir como prueba de estres",
        "justification": "Cuantifica degradacion ante ruido en variables sensibles.",
    },
    {
        "method_group": "diagnose_reliability / slicing_reliability",
        "status": "RECOMENDADO",
        "decision": "Incluir como control complementario",
        "justification": "Aporta una vista adicional de estabilidad de prediccion por segmentos.",
    },
    {
        "method_group": "diagnose_slicing_accuracy / overfit",
        "status": "RECOMENDADO",
        "decision": "Incluir en reporte de validacion",
        "justification": "Hace visible donde el modelo rinde peor por cortes de negocio.",
    },
    {
        "method_group": "detect_outlier_isolation_forest",
        "status": "RECOMENDADO",
        "decision": "Usar como indicador de calidad de entrada",
        "justification": "Ayuda a monitorear estabilidad de poblacion y datos anomales.",
    },
    {
        "method_group": "explain_pfi / explain_shap / explain_lime",
        "status": "RECOMENDADO",
        "decision": "Mantener para auditoria y explicabilidad",
        "justification": "Facilita interpretacion de drivers globales y casos individuales.",
    },
    {
        "method_group": "ModelZoo para reentrenar modelos alternos",
        "status": "DESCARTAR (por ahora)",
        "decision": "No usar para reemplazar pipeline core",
        "justification": "En este benchmark no supero al modelo canonico en AUC.",
    },
    {
        "method_group": "interpret_local_ei (logistic wrapper)",
        "status": "DESCARTAR (bug de libreria)",
        "decision": "No usar hasta correccion upstream",
        "justification": "Falla por atributo faltante (`modeva_intercept_`).",
    },
    {
        "method_group": "transform / inverse_transform con binning",
        "status": "DESCARTAR (inestable)",
        "decision": "Evitar en flujo productivo",
        "justification": "Se observaron errores de shape tras pipeline de binning.",
    },
    {
        "method_group": "eda_umap / visualizaciones pesadas",
        "status": "DESCARTAR (core)",
        "decision": "Usar solo para analisis exploratorio",
        "justification": "Valor operativo bajo para gate de produccion.",
    },
]

decision_df = pd.DataFrame(method_decisions)
display(decision_df)

core_recommended = decision_df[decision_df["status"].str.contains("RECOMENDADO")]
core_discarded = decision_df[~decision_df["status"].str.contains("RECOMENDADO")]

display(Markdown("### Lista corta para el proyecto principal"))
display(core_recommended[["method_group", "decision"]])

display(Markdown("### Lista a descartar/no priorizar en el core"))
display(core_discarded[["method_group", "decision"]])

,method_group,status,decision,justification
0,data_drift_test (PSI/KS),RECOMENDADO,Incluir como monitoreo periodico,Entrega alerta temprana de cambio distribucion...
1,diagnose_fairness / diagnose_slicing_fairness,RECOMENDADO,Incluir como control de gobernanza,Detecta sesgos por subgrupo y soporta discusio...
2,diagnose_robustness / diagnose_slicing_robustness,RECOMENDADO,Incluir como prueba de estres,Cuantifica degradacion ante ruido en variables...
3,diagnose_reliability / slicing_reliability,RECOMENDADO,Incluir como control complementario,Aporta una vista adicional de estabilidad de p...
4,diagnose_slicing_accuracy / overfit,RECOMENDADO,Incluir en reporte de validacion,Hace visible donde el modelo rinde peor por co...
5,detect_outlier_isolation_forest,RECOMENDADO,Usar como indicador de calidad de entrada,Ayuda a monitorear estabilidad de poblacion y ...
6,explain_pfi / explain_shap / explain_lime,RECOMENDADO,Mantener para auditoria y explicabilidad,Facilita interpretacion de drivers globales y ...
7,ModelZoo para reentrenar modelos alternos,DESCARTAR (por ahora),No usar para reemplazar pipeline core,En este benchmark no supero al modelo canonico...
8,interpret_local_ei (logistic wrapper),DESCARTAR (bug de libreria),No usar hasta correccion upstream,Falla por atributo faltante (`modeva_intercept...
9,transform / inverse_transform con binning,DESCARTAR (inestable),Evitar en flujo productivo,Se observaron errores de shape tras pipeline d...


### Lista corta para el proyecto principal

,method_group,decision
0,data_drift_test (PSI/KS),Incluir como monitoreo periodico
1,diagnose_fairness / diagnose_slicing_fairness,Incluir como control de gobernanza
2,diagnose_robustness / diagnose_slicing_robustness,Incluir como prueba de estres
3,diagnose_reliability / slicing_reliability,Incluir como control complementario
4,diagnose_slicing_accuracy / overfit,Incluir en reporte de validacion
5,detect_outlier_isolation_forest,Usar como indicador de calidad de entrada
6,explain_pfi / explain_shap / explain_lime,Mantener para auditoria y explicabilidad


### Lista a descartar/no priorizar en el core

,method_group,decision
7,ModelZoo para reentrenar modelos alternos,No usar para reemplazar pipeline core
8,interpret_local_ei (logistic wrapper),No usar hasta correccion upstream
9,transform / inverse_transform con binning,Evitar en flujo productivo
10,eda_umap / visualizaciones pesadas,Usar solo para analisis exploratorio
